In [1]:
from aequitas_algo.algo import run_aequitas
from data_generator.main2 import generate_data
import pandas as pd


def run_algo(model_type, min_number_of_classes=2, max_number_of_classes=6, nb_attributes=6,
             prop_protected_attr=0.3, nb_elems=500, hiddenlayers_depth=3, min_similarity=0.0,
             max_similarity=1.0, min_alea_uncertainty=0.0, max_alea_uncertainty=1.0,
             min_epis_uncertainty=0.0, max_epis_uncertainty=1.0,
             min_magnitude=0.0, max_magnitude=1.0, min_frequency=0.0, max_frequency=1.0,
             categorical_outcome=True, nb_categories_outcome=4,
             global_iteration_limit=1000, local_iteration_limit=100):
    modelst = ["DecisionTree", 'DecisionTreeClassifier', "MLPC", 'MLPClassifier', "SVM", 'SVC', "RandomForest",
               'RandomForestClassifier']

    model_type = modelst[model_type]
    df, protected_attr = generate_data(min_number_of_classes=min_number_of_classes, max_number_of_classes=max_number_of_classes, nb_attributes=nb_attributes,
                                       prop_protected_attr=prop_protected_attr, nb_elems=nb_elems, hiddenlayers_depth=hiddenlayers_depth, min_similarity=min_similarity,
                                       max_similarity=max_similarity, min_alea_uncertainty=min_alea_uncertainty, max_alea_uncertainty=max_alea_uncertainty,
                                       min_epis_uncertainty=min_epis_uncertainty, max_epis_uncertainty=max_epis_uncertainty,
                                       min_magnitude=min_magnitude, max_magnitude=max_magnitude, min_frequency=min_frequency, max_frequency=max_frequency,
                                       categorical_outcome=categorical_outcome, nb_categories_outcome=nb_categories_outcome)

    dff = df[[e for e in protected_attr] + ['outcome']]
    results_df = run_aequitas(dff, col_to_be_predicted="outcome",
                              sensitive_param_name_list=[k for k, e in protected_attr.items() if e],
                              perturbation_unit=1, model_type=model_type, threshold=0,
                              global_iteration_limit=global_iteration_limit, local_iteration_limit=local_iteration_limit)

    def generate_merged_dataframe(results_df, df, protected_attr):
        def transform_subgroup(x, protected_attr):
            res = list(map(lambda e: '|'.join(list(map(str, e))), x[list(protected_attr)].values.tolist()))
            res = ["*".join(res), "*".join(res[::-1])]
            return pd.Series(res, index=x.index)

        # Creating individual discrimination keys
        results_df['ind_discr_key'] = results_df.apply(
            lambda x: '|'.join(list(map(str, x[list(protected_attr)].values.tolist()))), axis=1)
        df['ind_discr_key'] = df.apply(lambda x: '|'.join(list(map(str, x[list(protected_attr)].values.tolist()))),
                                       axis=1)

        # Creating coupled discrimination keys
        results_df['couple_discr_key'] = results_df.groupby(['subgroup_num']).apply(
            lambda x: transform_subgroup(x, protected_attr)).reset_index(level=0, drop=True)
        df['couple_discr_key'] = df.groupby(['subgroup_num', 'subgroup_id']).apply(
            lambda x: transform_subgroup(x, protected_attr)).reset_index(level=0, drop=True).reset_index(level=0,
                                                                                                         drop=True)

        # Define columns for merging and results
        common_columns = ['diff_outcome', 'intersectionality', 'similarity', 'alea_uncertainty', 'epis_uncertainty',
                          'magnitude', 'frequency']
        results_columns = ['ind_discr_key', 'couple_discr_key', 'diff_outcome']

        # Merging for individual discrimination keys
        results_df_ind = results_df[results_columns].drop_duplicates().reset_index(drop=True)
        df2_ind = df[results_columns[:-1] + common_columns].drop_duplicates().reset_index(drop=True)
        merged_df_leftjoin_df2_ind = df2_ind.merge(results_df_ind, on='ind_discr_key', how='inner',
                                                   suffixes=('_injected', '_found'))
        merged_df_leftjoin_df2_ind['source'] = 'true_positive'

        # Merging for coupled discrimination keys
        results_df_couple = results_df[results_columns].drop_duplicates().reset_index(drop=True)
        df2_couple = df[results_columns[:-1] + common_columns].drop_duplicates().reset_index(drop=True)
        merged_df_leftjoin_df2_couple = df2_couple.merge(results_df_couple, on='couple_discr_key', how='inner',
                                                         suffixes=('_injected', '_found'))
        merged_df_leftjoin_df2_couple['source'] = 'true_positive'

        # Unique entries for individual keys
        df_unique_to_results_ind = pd.merge(results_df_ind[['ind_discr_key', 'diff_outcome']],
                                            df2_ind[['ind_discr_key']],
                                            on='ind_discr_key', how='left', indicator=True)
        df_unique_to_results_ind = df_unique_to_results_ind[df_unique_to_results_ind['_merge'] == 'left_only'].drop(
            columns=['_merge'])
        df_unique_to_results_ind = df_unique_to_results_ind.assign(**{col: None for col in common_columns[1:]})
        df_unique_to_results_ind['diff_outcome_found'] = df_unique_to_results_ind['diff_outcome']
        df_unique_to_results_ind['source'] = 'false_positive'

        # Unique entries for coupled keys
        df_unique_to_results_couple = pd.merge(results_df_couple[['couple_discr_key', 'diff_outcome']],
                                               df2_couple[['couple_discr_key']], on='couple_discr_key', how='left',
                                               indicator=True)
        df_unique_to_results_couple = df_unique_to_results_couple[
            df_unique_to_results_couple['_merge'] == 'left_only'].drop(columns=['_merge'])
        df_unique_to_results_couple = df_unique_to_results_couple.assign(**{col: None for col in common_columns[1:]})
        df_unique_to_results_couple['diff_outcome_found'] = df_unique_to_results_couple['diff_outcome']
        df_unique_to_results_couple['source'] = 'false_positive'

        indv_df = pd.concat([merged_df_leftjoin_df2_ind, df_unique_to_results_ind], ignore_index=True)
        couple_df = pd.concat([merged_df_leftjoin_df2_couple, df_unique_to_results_couple], ignore_index=True)

        return df, results_df, couple_df, indv_df

    df, result_df, couple_df, indv_df = generate_merged_dataframe(results_df, df, protected_attr)

    def calculate_metrics(df, results_df, couple_df):
        unique_keys_in_df = df[
            'couple_discr_key'].nunique()  # Adjust 'ind_discr_key' to the correct column name if necessary
        unique_keys_in_results_df = results_df[
            'couple_discr_key'].nunique()  # Adjust 'ind_discr_key' to the correct column name if necessary

        # Calculate TPR and FPR using the unique keys count
        tpr = couple_df[couple_df['source'] == 'true_positive'].shape[0] / unique_keys_in_df
        fpr = couple_df[couple_df['source'] == 'false_positive'].shape[0] / unique_keys_in_results_df

        return tpr, fpr

    tpr, fpr = calculate_metrics(df, results_df, couple_df)

    return tpr, fpr


In [4]:
tpr, fpr = run_algo(0, min_number_of_classes=2, max_number_of_classes=6, nb_attributes=6,
             prop_protected_attr=0.3, nb_elems=500, hiddenlayers_depth=3, min_similarity=0.0,
             max_similarity=1.0, min_alea_uncertainty=0.0, max_alea_uncertainty=1.0,
             min_epis_uncertainty=0.0, max_epis_uncertainty=1.0,
             min_magnitude=0.0, max_magnitude=1.0, min_frequency=0.0, max_frequency=1.0,
             categorical_outcome=True, nb_categories_outcome=4,
             global_iteration_limit=1000, local_iteration_limit=100)

print(f"{tpr}-{fpr}")

granularity 4, intersectionality 1, similarity 0.23061045736897845 maxed !
granularity 5, intersectionality 1, similarity 0.4273787402711393 maxed !
granularity 3, intersectionality 1, similarity 0.8332253527841518 maxed !
granularity 2, intersectionality 1, similarity 0.6384969212051725 maxed !
granularity 1, intersectionality 1, similarity 0.3371264753513714 maxed !
granularity 1, intersectionality 1, similarity 0.23210287963592624 maxed !
granularity 1, intersectionality 1, similarity 0.9561871713999994 maxed !
granularity 2, intersectionality 1, similarity 0.3643605102604818 maxed !
granularity 4, intersectionality 1, similarity 0.312940571136482 maxed !
granularity 3, intersectionality 1, similarity 0.023146397336419544 maxed !
granularity 3, intersectionality 1, similarity 0.35819391350908913 maxed !
granularity 2, intersectionality 1, similarity 0.5176827047361781 maxed !
granularity 5, intersectionality 1, similarity 0.440988817320377 maxed !
granularity 5, intersectionality 1,

/home/vaunorage/PycharmProjects/ClearBias/data_generator/main2.py:128: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[column] = pd.to_numeric(df[column], errors='ignore')
/home/vaunorage/PycharmProjects/ClearBias/venv/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


{'model': 'DecisionTreeClassifier', 'score': 0.49294655275183785, 'f1_score': 0.41367577848155496}
Aequitas Fully Directed Started for Attr1_T...

Finished Global Search
Local Search Finished


/tmp/ipykernel_717312/716375537.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_df['couple_discr_key'] = results_df.groupby(['subgroup_num']).apply(


0.16796875-0.0


/tmp/ipykernel_717312/716375537.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['couple_discr_key'] = df.groupby(['subgroup_num', 'subgroup_id']).apply(
/tmp/ipykernel_717312/716375537.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  indv_df = pd.concat([merged_df_leftjoin_df2_ind, df_unique_to_results_ind], ignore_index=True)
/tmp/ipykernel_717312/716375537.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA

In [2]:
import numpy as np
import sqlite3
import random

def run_algo_simulated(**params):
    tpr = random.random()  # Simulate a result for True Positive Rate
    fpr = random.random()  # Simulate a result for False Positive Rate
    return tpr, fpr

def generate_parameters(parameter_bounds):
    params = {}
    for key, (low, high) in parameter_bounds.items():
        if isinstance(low, bool):
            params[key] = low  # Assuming both low and high are True for bools
        elif isinstance(low, int):
            params[key] = random.randint(low, high)
        else:
            params[key] = random.uniform(low, high)
    return params

def create_database():
    conn = sqlite3.connect('experiment_results.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS results (
            id INTEGER PRIMARY KEY,
            tpr REAL,
            fpr REAL,
            parameters TEXT
        )
    ''')
    conn.commit()
    conn.close()

def save_results(tpr, fpr, params):
    conn = sqlite3.connect('experiment_results.db')
    c = conn.cursor()
    c.execute('''
        INSERT INTO results (tpr, fpr, parameters) VALUES (?, ?, ?)
    ''', (tpr, fpr, str(params)))
    conn.commit()
    conn.close()    



In [3]:
parameter_bounds = {
    "model_type":(0, 7),
        "mean_number_of_classes": (2, 10),
        "std_number_of_classes": (2, 10),
        "nb_attributes": (1, 30),
        "prop_protected_attr": (0.0, 1.0),
        "nb_elems": (200, 10000),
        "hiddenlayers_depth": (1, 6),
        "min_similarity": (0.0, 1.0),
        "max_similarity": (0.0, 1.0),
        "min_alea_uncertainty": (0.0, 1.0),
        "max_alea_uncertainty": (0.0, 1.0),
        "min_epis_uncertainty": (0.0, 1.0),
        "max_epis_uncertainty": (0.0, 1.0),
        "min_magnitude": (0.0, 1.0),
        "max_magnitude": (0.0, 1.0),
        "min_frequency": (0.0, 1.0),
        "max_frequency": (0.0, 1.0),
        "categorical_outcome": (True, True),  # assuming always True
        "nb_categories_outcome": (1, 24),
        "global_iteration_limit": (1, 10000),
        "local_iteration_limit": (1, 1000)
    }
    
create_database()
k = 10 

for _ in range(k):
    params = generate_parameters(parameter_bounds)
    nb_classes = random.randint(params['mean_number_of_classes']-params['std_number_of_classes'],
                                params['mean_number_of_classes']+params['std_number_of_classes'])
    params['min_number_of_classes']=nb_classes-1
    params['max_number_of_classes']=nb_classes+2
    del params['mean_number_of_classes']
    del params['std_number_of_classes']
    print(params)
    tpr, fpr = run_algo(**params)
    print(tpr, fpr)
    save_results(tpr, fpr, params)

{'model_type': 0, 'nb_attributes': 20, 'prop_protected_attr': 0.5588089917613975, 'nb_elems': 2254, 'hiddenlayers_depth': 1, 'min_similarity': 0.40638647154171326, 'max_similarity': 0.0226894658326815, 'min_alea_uncertainty': 0.2073243606573132, 'max_alea_uncertainty': 0.5167842961957001, 'min_epis_uncertainty': 0.8283251340191496, 'max_epis_uncertainty': 0.5920511678120483, 'min_magnitude': 0.3180409093624176, 'max_magnitude': 0.6600760397871223, 'min_frequency': 0.9578986215739218, 'max_frequency': 0.6399528173765562, 'categorical_outcome': True, 'nb_categories_outcome': 3, 'global_iteration_limit': 4595, 'local_iteration_limit': 363, 'min_number_of_classes': 1, 'max_number_of_classes': 4}


ValueError: not enough values to unpack (expected 2, got 0)